# 🔐 Login Produttivo - Captura de Cookie
**Versão simplificada para execução via API**

In [ ]:
# Instalação de dependências
!pip install playwright -q
!playwright install chromium
!playwright install-deps chromium

In [ ]:
from playwright.async_api import async_playwright
import asyncio
from typing import Optional

# Configurações
URL_LOGIN = "https://app.produttivo.com.br/auth/sign_in"

# Variável global para cookie
produttivo_session_cookie: Optional[str] = None

In [ ]:
def extrair_cookie_produttivo(cookie_header: str) -> Optional[str]:
    """Extrai apenas o cookie _produttivo_session"""
    if cookie_header:
        pares = cookie_header.split('; ')
        for par in pares:
            if '=' in par:
                nome, valor = par.split('=', 1)
                if nome == '_produttivo_session':
                    return valor
    return None

async def capturar_request_works(request):
    """Intercepta requisição /works e captura cookie"""
    global produttivo_session_cookie
    
    if request.url == "https://app.produttivo.com.br/works" and request.method == "GET":
        print("🎯 Requisição /works capturada!")
        
        headers = await request.all_headers()
        cookie_header = headers.get('cookie', '')
        
        if cookie_header:
            produttivo_session_cookie = extrair_cookie_produttivo(cookie_header)
            if produttivo_session_cookie:
                print("✅ Cookie _produttivo_session capturado!")

In [ ]:
async def fazer_login(email: str, senha: str) -> Optional[str]:
    """Executa login e retorna cookie"""
    global produttivo_session_cookie
    produttivo_session_cookie = None
    
    print("🔐 Iniciando login...")
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-dev-shm-usage']
        )
        
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36'
        )
        
        context.set_default_timeout(60000)
        page = await context.new_page()
        page.on("request", capturar_request_works)
        
        print(f"🌐 Acessando: {URL_LOGIN}")
        
        try:
            await page.goto(URL_LOGIN, wait_until="domcontentloaded", timeout=60000)
            print("✓ Página carregada")
            await page.wait_for_timeout(3000)
            
            print("🔐 Preenchendo credenciais...")
            
            await page.wait_for_selector('input[type="email"], input[name="email"]', timeout=30000)
            await page.fill('input[type="email"], input[name="email"]', email)
            print("✓ E-mail preenchido")
            
            await page.wait_for_timeout(1000)
            await page.fill('input[type="password"]', senha)
            print("✓ Senha preenchida")
            
            await page.wait_for_timeout(2000)
            
            # Tenta clicar no botão
            try:
                await page.click('button:has-text("Login")', timeout=5000)
            except:
                try:
                    await page.click('button[type="submit"]', timeout=5000)
                except:
                    await page.press('input[type="password"]', 'Enter')
            
            print("✓ Login enviado")
            print("⏳ Aguardando autenticação...")
            
            await page.wait_for_timeout(8000)
            
            if page.url != URL_LOGIN:
                print("✅ LOGIN BEM-SUCEDIDO!")
                await page.wait_for_timeout(2000)
                
                await browser.close()
                return produttivo_session_cookie
            else:
                print("❌ Login falhou")
                await browser.close()
                return None
                
        except Exception as e:
            print(f"❌ ERRO: {e}")
            await browser.close()
            return None

In [ ]:
# EXECUÇÃO
# Estas variáveis serão injetadas pela API
EMAIL = "financeiro@rfsolucoestelecom.com.br"  # Será substituído
SENHA = "Novo*789"  # Será substituído

# Executa login
cookie = await fazer_login(EMAIL, SENHA)

if cookie:
    print("\n" + "="*70)
    print("🍪 COOKIE CAPTURADO:")
    print("="*70)
    print(cookie)
    print("="*70)
else:
    print("\n❌ Falha ao capturar cookie")